In [158]:
import pandas as pd
import lxml
import glob
from lxml import etree
from bs4 import BeautifulSoup
import re 
import requests
from urllib.parse import quote
import sys
sys.stdout = open('output.txt','wt')

ns = "{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}"
cards = './urdu_cards_pages/1032971/U2_for_Transkribus/page/*.xml'

In [159]:

for card in glob.glob(cards):
    with open(card, "r") as file:
        soup = BeautifulSoup(file, 'xml')
        shelfmark = soup.select('TextRegion[custom*=shelfmark]')[0].find_all('TextLine', recursive=False)[0].find_all('TextEquiv', recursive=False)[0].find_all('Unicode', recursive=False)[0]
        shelfmark = shelfmark.get_text()
        author = soup.select('TextRegion[custom*=author]')[0].find_all('TextLine', recursive=False)[0].find_all('TextEquiv', recursive=False)[0].find_all('Unicode', recursive=False)[0]
        author = author.get_text()
        title = soup.select('TextRegion[custom*=title]')[0].find_all('TextLine', recursive=False)[0].find_all('TextEquiv', recursive=False)[0].find_all('Unicode', recursive=False)[0]
        title = title.get_text()


In [160]:
surname = author.split(',')[0]
surname_encoded = quote(surname)

#url = 'https://www.worldcat.org/search?q=ti%3Apromessi+au%3Amanzoni&qt=advanced&dblist=638'
url = 'https://www.worldcat.org/search?q=au%3A'+surname_encoded+'&qt=advanced&dblist=638'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')


In [161]:
results_info = int(soup.select('div[class=resultsinfo]')[0].find_all('td')[0].find_all('strong')[1].get_text())
n_pages = results_info//10


In [162]:
urls = [url]  
if n_pages > 1:
    for i in range(1,n_pages+1):
        urls.append(url+'&start='+str(i*10+1)+'&qt=page_number_link')
urls

['https://www.worldcat.org/search?q=au%3ABARNI%20%28Muhammad%20Ilyas%29&qt=advanced&dblist=638',
 'https://www.worldcat.org/search?q=au%3ABARNI%20%28Muhammad%20Ilyas%29&qt=advanced&dblist=638&start=11&qt=page_number_link',
 'https://www.worldcat.org/search?q=au%3ABARNI%20%28Muhammad%20Ilyas%29&qt=advanced&dblist=638&start=21&qt=page_number_link',
 'https://www.worldcat.org/search?q=au%3ABARNI%20%28Muhammad%20Ilyas%29&qt=advanced&dblist=638&start=31&qt=page_number_link',
 'https://www.worldcat.org/search?q=au%3ABARNI%20%28Muhammad%20Ilyas%29&qt=advanced&dblist=638&start=41&qt=page_number_link',
 'https://www.worldcat.org/search?q=au%3ABARNI%20%28Muhammad%20Ilyas%29&qt=advanced&dblist=638&start=51&qt=page_number_link',
 'https://www.worldcat.org/search?q=au%3ABARNI%20%28Muhammad%20Ilyas%29&qt=advanced&dblist=638&start=61&qt=page_number_link',
 'https://www.worldcat.org/search?q=au%3ABARNI%20%28Muhammad%20Ilyas%29&qt=advanced&dblist=638&start=71&qt=page_number_link']

In [163]:
print('===========')
print("CARD TRANSCRIPTION")
print('Shelfmark = '+ shelfmark)
print('Author = '+ author)
print('Title = '+ title)
print()
print("**PARAMETERS USED ON WORLDCAT**")
print('Surname: ' + surname)
print('N. results: ' + str(results_info))
print('===========')
print()
print("RESULTS:")

n = 1

for u in urls:
    page = requests.get(u)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.select('a[id*=result]')
    for result in results:
        s = str(result['href'])
        print(n)
        title = result.find('strong').text  
        print('Title: '+ title)
        ocloc_id = re.search(r'[0-9]{6,}', s)
        print('ocloc_id: '+ ocloc_id.group(0))
        url = 'http://www.worldcat.org/oclc/'+ocloc_id.group(0)
        print('URL: '+ url)
        print('===========')
        print()
        n +=1
